In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split

from scripts.pipeline import Pipeline

In [11]:
# Load the data
train_data = pd.read_csv('../data/train.csv')
test_data = pd.read_csv('../data/test.csv')

# split train data into predictors and response
X = train_data.drop(columns=['SalePrice'])
y = train_data['SalePrice'] 

#combine the data for preprocessing
data = pd.concat([X, test_data], axis=0)
# Preprocess the data
data = Pipeline(data)

# Split the data back into train and test, processed
X = data[:len(train_data)]
test = data[len(train_data):]
#increase test index by 1
test.index += 1

# Split the data into features and response
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
#going with extra trees regressor
from sklearn.ensemble import ExtraTreesRegressor
model = ExtraTreesRegressor()
model.fit(X_train, y_train)

# Predict the test data
y_pred = model.predict(test)
test_results = pd.DataFrame({'Id': test.index, 'SalePrice': y_pred})
test_results.to_csv('../outputs/ExtraTreesRegressor_test.csv', index=False)

In [13]:
#going with gradient boosting regressor
from sklearn.ensemble import GradientBoostingRegressor
model = GradientBoostingRegressor()
model.fit(X_train, y_train)

# Predict the test data
y_pred = model.predict(test)
test_results = pd.DataFrame({'Id': test.index, 'SalePrice': y_pred})
test_results.to_csv('../outputs/GradientBoostingRegressor_test.csv', index=False)